In [1]:
#%pip install requests lxml pandas
#%pip install --upgrade ipykernel
#%pip install bs4

Парсинг информации с сайта с использованием xpath

In [ ]:
#Импортируем необходимые библиотеки
from pprint import pprint
from lxml import html
import requests
import pandas as pd

#Указываем ссылку на сайт
url = "https://www.old-games.ru/catalog/?platform=1&sort=popularity&ysclid=mgtig0uzj8331895606"

r = requests.get(url)
if r.status_code != 200:
    print(f"Ошибка загрузки страницы: {r.status_code}")
    exit()

root = html.fromstring(r.content)

rows = root.xpath('//table[contains(@class,"listtable")]//tr[contains(@id,"game_")]')

data = []

for row in rows:
    # td элементы внутри строки
    tds = row.xpath('./td')
    if len(tds) < 6:
        continue

    # Название — в первом td, первый <a>
    name = tds[0].xpath('.//a[1]/text()')
    name_text = name[0] if name else 'N/A'

    # Жанр — во втором td, ссылка или текст
    genre = tds[1].xpath('.//a/text()')
    genre_text = genre[0] if genre else tds[1].text_content().strip()

    # Год — в третьем td, ссылка или текст
    year = tds[2].xpath('.//a/text()')
    year_text = year[0] if year else tds[2].text_content().strip()

    # Платформа — в четвертом td, ссылка или текст
    platform = tds[3].xpath('.//a/text()')
    platform_text = platform[0] if platform else tds[3].text_content().strip()

    # Издатель — в пятом td, ссылка или текст
    publisher = tds[4].xpath('.//a/text()')
    publisher_text = publisher[0] if publisher else tds[4].text_content().strip()

    # Оценка — alt атрибут img в шестом td
    rating_img_alt = tds[5].xpath('.//img/@alt')
    rating_text = rating_img_alt[0] if rating_img_alt else 'N/A'

    data.append({
        'Название': name_text,
        'Жанр': genre_text,
        'Год': year_text,
        'Платформа': platform_text,
        'Издатель': publisher_text,
        'Оценка': rating_text
    })

df = pd.DataFrame(data)
print(df.to_markdown(index=False))



| Название                                                | Жанр      |   Год | Платформа   | Издатель                        | Оценка                       |
|:--------------------------------------------------------|:----------|------:|:------------|:--------------------------------|:-----------------------------|
| WarCraft II: Tides of Darkness                          | Strategy  |  1995 | DOS         | Blizzard Entertainment          | Оценка рецензента - 10 из 10 |
| Blood                                                   | Action    |  1997 | DOS         | GT Interactive Software         | Оценка рецензента - 10 из 10 |
| Quake                                                   | Action    |  1996 | DOS         | id Software                     | Оценка рецензента - 10 из 10 |
| X-COM: UFO Defense                                      | Strategy  |  1994 | DOS         | MicroProse Software             | Оценка рецензента - 8 из 10  |
| DOOM                                        

Парсинг информации с сайта с использованием BeautifulSoup

In [ ]:
#Импортируем необходимые библиотеки
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Указываем ссылку на сайт
url = "https://www.old-games.ru/catalog/?platform=1&sort=popularity&ysclid=mgtig0uzj8331895606"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
}

r = requests.get(url, headers=headers, timeout=30)
r.raise_for_status()
r.encoding = r.apparent_encoding

soup = BeautifulSoup(r.text, "lxml")

table = soup.select_one("table.listtable")
rows = table.select('tr[id^="game_"]')

data = []

for tr in rows:
    # берем прямые <td>
    tds = tr.find_all("td", recursive=False)
    if len(tds) < 6:
        continue

    # Название — в первом td, первый <a>
    name_a = tds[0].find("a", href=True)
    name_text = name_a.get_text(strip=True) if name_a else tds[0].get_text(strip=True)

    # Жанр — во втором td, ссылка или текст
    genre_links = [a.get_text(strip=True) for a in tds[1].find_all("a")]
    genre_text = ", ".join(genre_links) if genre_links else tds[1].get_text(strip=True)

    # Год — в третьем td, ссылка или текст
    year_a = tds[2].find("a")
    year_text = year_a.get_text(strip=True) if year_a else tds[2].get_text(strip=True)

    # Платформа — в четвертом td, ссылка или текст
    platform_a = tds[3].find("a")
    platform_text = platform_a.get_text(strip=True) if platform_a else tds[3].get_text(strip=True)

    # Издатель — в пятом td, ссылка или текст
    publisher_a = tds[4].find("a")
    publisher_text = publisher_a.get_text(strip=True) if publisher_a else tds[4].get_text(strip=True)

    # Оценка — alt атрибут img в шестом td
    rating_img = tds[5].find("img")
    rating_text = rating_img.get("alt", "N/A") if rating_img else "N/A"

    data.append({
        "Название": name_text,
        "Жанр": genre_text,
        "Год": year_text,
        "Платформа": platform_text,
        "Издатель": publisher_text,
        "Оценка": rating_text
    })

df = pd.DataFrame(data)
print(df.to_markdown(index=False))


| Название                                                | Жанр      |   Год | Платформа   | Издатель                        | Оценка                       |
|:--------------------------------------------------------|:----------|------:|:------------|:--------------------------------|:-----------------------------|
| WarCraft II: Tides of Darkness                          | Strategy  |  1995 | DOS         | Blizzard Entertainment          | Оценка рецензента - 10 из 10 |
| Blood                                                   | Action    |  1997 | DOS         | GT Interactive Software         | Оценка рецензента - 10 из 10 |
| Quake                                                   | Action    |  1996 | DOS         | id Software                     | Оценка рецензента - 10 из 10 |
| X-COM: UFO Defense                                      | Strategy  |  1994 | DOS         | MicroProse Software             | Оценка рецензента - 8 из 10  |
| DOOM                                        